<a href="https://colab.research.google.com/github/JordanFoss/STAT3007_Project/blob/main/Encoder_logistic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/JordanFoss/STAT3007_Project.git
%cd STAT3007_Project/

Cloning into 'STAT3007_Project'...
remote: Enumerating objects: 3845, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 3845 (delta 46), reused 84 (delta 25), pack-reused 3722
Receiving objects: 100% (3845/3845), 675.27 MiB | 31.12 MiB/s, done.
Resolving deltas: 100% (631/631), done.
Checking out files: 100% (2830/2830), done.
/content/STAT3007_Project


In [4]:
import librosa
from librosa import display
import colorednoise as cn
import torch
import numpy as np
from IPython.display import Audio
import glob
from pre_process import *
import os
import torch.optim as optim
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import ToTensor

import scipy
from torchsummary import summary


In [ ]:

spectro_data = DatasetWrapper(X,y)
train_size = int(X.shape[0] * 0.7)
test_size = X.shape[0] - train_size

loss = nn.MSELoss()

data_test, data_train = random_split(spectro_data,[test_size,train_size], generator = torch.Generator().manual_seed(10))
nepoch = 100
batch_size = 30

torch.manual_seed(69)
AE = Autoencoder(16)

trained_net = train_model(data_train,AE, loss, nepoch = nepoch,lr = 0.01, batch_size = batch_size, use_cuda = True,print_output = True)

# next step: use encoder to classify

In [14]:
# load the train feature net (encoder):
feature_net = trained_net.get_encoder()

In [15]:
summary(feature_net.cuda(),(1,128,63))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 126, 61]             160
              ReLU-2          [-1, 16, 126, 61]               0
         MaxPool2d-3           [-1, 16, 63, 30]               0
              ReLU-4           [-1, 16, 63, 30]               0
            Conv2d-5           [-1, 16, 61, 28]           2,320
              ReLU-6           [-1, 16, 61, 28]               0
         MaxPool2d-7           [-1, 16, 30, 14]               0
            Conv2d-8           [-1, 16, 28, 12]           2,320
              ReLU-9           [-1, 16, 28, 12]               0
Total params: 4,800
Trainable params: 4,800
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 2.89
Params size (MB): 0.02
Estimated Total Size (MB): 2.94
-----------------------------------------------

In [29]:


# input: 128xtime_steps
#output: 30x14 for 32 filters
class ConvNet(nn.Module):
    def __init__(self, input_channel,contain_linear = False, filter_num = 16):
        super(ConvNet, self).__init__()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(input_channel, filter_num, kernel_size = (2,3))
        self.conv2 = nn.Sequential(nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1,2), stride = 2),
            nn.Dropout(0.25),
            nn.Conv2d(filter_num, 8, kernel_size = (2,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(1,2), stride = 2),
        )

        self.contain_linar = contain_linear

        if contain_linear:
          self.linear = nn.Sequential(
              nn.Linear(24*7, 1024),
              nn.Linear(1024, 5),
          )

    def forward(self, x, inspect_feature = False):

      first_layer = self.conv1(x)
      conv_x = self.conv2(first_layer)

      output_x = conv_x
      if self.contain_linar:
        conv_x_flat  = self.flatten(conv_x)
        output_x = self.linear(conv_x_flat)
      
      if inspect_feature:
        return first_layer,conv_x,output_x
      return output_x

In [30]:
summary(ConvNet(16).cuda(),(16,28,12))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 27, 10]           1,552
              ReLU-2           [-1, 16, 27, 10]               0
         MaxPool2d-3            [-1, 16, 14, 5]               0
           Dropout-4            [-1, 16, 14, 5]               0
            Conv2d-5             [-1, 8, 13, 3]             776
              ReLU-6             [-1, 8, 13, 3]               0
         MaxPool2d-7              [-1, 8, 7, 1]               0
Total params: 2,328
Trainable params: 2,328
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.02
Forward/backward pass size (MB): 0.09
Params size (MB): 0.01
Estimated Total Size (MB): 0.12
----------------------------------------------------------------


In [31]:
Xc = []
yc = []
target_map = {'02':0,'03':1,'04':2,'05':3,'08':4}

for file in glob.glob(os.getcwd() + '/sample-noisy-speech-actor-11/*.wav'):
  name = file.split('/')[-1]
  emotion = name.split('-')[0]

  samples, sampling_rate = librosa.load(file, sr = 16000)
  spec = data_gen(samples, sampling_rate, 2)

  Xc.append(spec)
  yc.append(target_map[emotion])

Xc = torch.tensor(Xc)
Xc = Xc.reshape(Xc.shape[0],1,Xc.shape[1],Xc.shape[2])

X_feature = feature_net.to(torch.device('cpu'))(Xc)

yc = torch.tensor(yc)
yc = yc.type(torch.LongTensor)

In [27]:
X_feature = torch.tensor(X_feature.detach().numpy())

In [28]:
classification_data = DatasetWrapper(X_feature,yc)
train_size = int(X_feature.shape[0] * 0.7)
test_size = X_feature.shape[0] - train_size

loss = nn.CrossEntropyLoss()

data_test, data_train = random_split(classification_data,[test_size,train_size], generator = torch.Generator().manual_seed(10))
nepoch = 100
batch_size = 30

torch.manual_seed(69)
CNN = ConvNet(input_channel = 16, contain_linear= True).cuda()



trained_net = train_model(data_train,CNN, loss, nepoch = nepoch,lr = 0.01, batch_size = batch_size, use_cuda = True,print_output = True, classification = True)

epoch: 0
loss: 6.794913929297763e-07
------------
epoch: 1
loss: 0.6743510961532593
------------
epoch: 2
loss: 0.0003673128376249224
------------
epoch: 3
loss: 0.0
------------
epoch: 4
loss: 0.0
------------
epoch: 5
loss: 0.0
------------
epoch: 6
loss: 12.359458923339844
------------
epoch: 7
loss: 0.0
------------
epoch: 8
loss: 16.698596954345703
------------
epoch: 9
loss: 0.0
------------
epoch: 10
loss: 72.20507049560547
------------
epoch: 11
loss: 0.0
------------
epoch: 12
loss: 0.0
------------
epoch: 13
loss: 0.0
------------
epoch: 14
loss: 0.0
------------
epoch: 15
loss: 0.0
------------
epoch: 16
loss: 0.0
------------
epoch: 17
loss: 0.0
------------
epoch: 18
loss: 0.0
------------
epoch: 19
loss: 0.0
------------
epoch: 20
loss: 0.0
------------
epoch: 21
loss: 0.0
------------
epoch: 22
loss: 0.0
------------
epoch: 23
loss: 0.0
------------
epoch: 24
loss: 0.0
------------
epoch: 25
loss: 0.0
------------
epoch: 26
loss: 0.0
------------
epoch: 27
loss: 0.0
----

In [32]:
X_feature.shape

torch.Size([790, 16, 28, 12])

In [48]:
X_feature_flat = torch.flatten(X_feature,start_dim = 1)
classification_data = DatasetWrapper(X_feature_flat,yc)
train_size = int(X_feature.shape[0] * 0.7)
test_size = X_feature.shape[0] - train_size

data_test, data_train = random_split(classification_data,[test_size,train_size], generator = torch.Generator().manual_seed(10))

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import numpy as np

# building a naive bayes classifier
def classification_model(X_train,y_train, X_test, y_test):
  NB = MultinomialNB()
  NB.fit(X_train,y_train)
  y_pred = NB.predict(X_test)

  # metrics
  # count correctly classified samples
  correct_count = np.count_nonzero(y_pred == y_test)
  accuracy = metrics.accuracy_score(y_test,y_pred)
  balanced_accuracy = metrics.balanced_accuracy_score(y_test,y_pred)

  print('total sample counts:',y_pred.shape[0])
  print('correct classification counts:',correct_count,'(',accuracy * 100,'%)')
  print('{0}: {1:.1%}'.format('balanced accuracy', balanced_accuracy))
  print('misclasification counts:',y_pred.shape[0] - correct_count,'(',(1 - accuracy)*100,'%)')

  return NB

In [54]:
training_pixel_feature,training_pixel_target = data_train.dataset.get_data()
test_pixel_feature,test_pixel_target = data_test.dataset.get_data()


NB_pixel = classification_model(training_pixel_feature.detach().numpy(),training_pixel_target.detach().numpy(),test_pixel_feature.detach().numpy(),test_pixel_target.detach().numpy())

total sample counts: 790
correct classification counts: 783 ( 99.1139240506329 %)
balanced accuracy: 99.1%
misclasification counts: 7 ( 0.8860759493670933 %)
